In [1]:
import torch
from tqdm.notebook import tqdm
import trimesh
import scipy
import numpy as np
from scipy.sparse.linalg import spsolve
import robust_laplacian

import plotly.io as pio

pio.renderers.default = "browser"

from plot import animate_mesh_frames
from shape import calc_tri_areas

from smplpytorch.pytorch.smpl_layer import SMPL_Layer
from display_utils import display_model

In [2]:
cuda = False
batch_size = 1

# Create the SMPL layer
smpl_layer = SMPL_Layer(
    center_idx=0,
    gender='male',
    model_root='smplpytorch/native/models')

model_faces = smpl_layer.th_faces

# Generate T-pose and random shape parameters
pose_params = torch.rand(batch_size, 72) * 0.

# matrix of shapes
total_shapes_matrix = []
partial_shapes_matrix = []
#limit = 30.0
limit = 45.0
#start_vertex_index = 6866
#shape_params = torch.Tensor([[-4.3178, 2.5392, 3.9581, -3.6548, -3.0813, -1.8783, -4.1754, 0.6651, -3.2443, -0.8823]])
start_vertex_index = 5907

num_shapes = 1000

In [7]:
total_shapes_matrix = []
partial_shapes_matrix = []
for _ in tqdm(range(num_shapes)):
    # Generate random shape parameters
    shape_params = torch.Tensor([np.random.uniform(-5., 5., 10).tolist()])
    
    if cuda:
        pose_params = pose_params.cuda()
        shape_params = shape_params.cuda()
        smpl_layer.cuda()
    
    # Forward from the SMPL layer
    verts, Jtr = smpl_layer(pose_params, th_betas=shape_params)
    
    # Append the total shape
    total_shapes_matrix.append(np.array(verts)[0, :, :])
    
    # Convert list of triangles in adjacency matrices
    adjacency, edges = trimesh.graph.face_adjacency(model_faces, return_edges=True)

    # Transform edges matrix into a sparse matrix
    coo = trimesh.graph.edges_to_coo(edges)
    csr = coo.tocsr()

    # Dijkstra Algorithm
    distances = scipy.sparse.csgraph.dijkstra(csr, directed=False)
    distances_from_start_vertex = distances[start_vertex_index]

    # border indices [POSITION in verts array], distance from start_vertex == limit
    border_indices = [i for i in range(len(distances_from_start_vertex)) if distances_from_start_vertex[i] == limit]
    
    # partial verts indices [POSITION in verts array] in the total shape, with distance from start_vertex <= limit
    partial_shape_indices = [i for i in range(len(distances_from_start_vertex)) if distances_from_start_vertex[i] <= limit]

    # partial verts position [VALUE in verts array] respect to the total shape
    partial_shape_verts = []
    for index in partial_shape_indices:
        partial_shape_verts.append(verts[0][index].numpy().tolist())
    
    # correspondence between indices in total/partial verts arrays
    verts_indices_corr = []
    for index in partial_shape_indices:
        verts_indices_corr.append([index, partial_shape_indices.index(index)])

    # faces containing partial verts indices
    partial_shape_faces = []
    for face in model_faces: # for all faces
        face_list = face.numpy().tolist()
        vertex_count = 0
        for index in partial_shape_indices: 
            if index in face_list:
                vertex_count = vertex_count + 1
                if vertex_count > 2: # if a face contains partial shape indices
                    partial_shape_faces.append(face_list) # append the face
                
    # map selected faces to selected vertices
    partial_shape_faces_mapped = [x[:] for x in partial_shape_faces]
    for i, face in enumerate(partial_shape_faces_mapped):
        for j, vertex_index in enumerate(face):
            # get the index position in "partial_shape_indices" of the vertex index
            partial_shape_faces_mapped[i][j] = partial_shape_indices.index(vertex_index)
    
    # select partial shape border indices [POSITION in partial_shape_verts] in total shape
    partial_shape_border_indices = []
    for l in verts_indices_corr:
        if l[0] in border_indices:
            partial_shape_border_indices.append(l[1])
            
    # Mean-Curvature Flow
    delta = 0.0001
    #max_iter = 50
    max_iter = 35

    # define new shape using 'partial_shape_verts' and 'partial_shape_faces_mapped'
    V = np.array(partial_shape_verts)
    F = np.array(partial_shape_faces_mapped)

    L, _ = robust_laplacian.mesh_laplacian(V, F)
    U = V
    flow = [(np.array(verts)[0, :, :], model_faces)]

    for i in range(max_iter):
        _, M = robust_laplacian.mesh_laplacian(U, F)

        A = M + delta*L
        A = A.tolil()
        B = M@U

        for j in range(A.shape[0]):
            if j in partial_shape_border_indices:
                for k in range(A.shape[1]):
                    A[j, k] = 1. if j == k else 0.
                B[j] = U[j]
            else:
                for k in range(A.shape[1]):
                    if k in partial_shape_border_indices:
                        B[j] = B[j] - U[j]*A[j,k]
                        A[j, k] = 0.

        A = A.tocsc()
        U = spsolve(A, B)

        full_verts = np.array(verts)[0, :, :]
        full_verts[np.array(verts_indices_corr)[:, 0]] = U
        flow.append((full_verts, model_faces))
        
    partial_shapes_matrix.append(full_verts)